In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.layers import Input, Flatten, Dense
from tensorflow.python.keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False) # data_format = "channels_first"

# Freeze the layers except the last 4 layers
for layer in model_vgg16_conv.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in model_vgg16_conv.layers:
    print(layer, layer.trainable)

#Create your own input format (here 3x200x200)
input = Input(shape=(150,150,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dense(1, activation='linear', name='pred')(x)

#Create your own model 
my_model = Model(input, x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training

my_model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse'])
my_model.summary()

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x13e67e290> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e67ebd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e67ec10> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x13e6b9d10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e6ee3d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e6f8f90> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x13e6ff590> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e704cd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e704490> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e712c90> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x13e71cc10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x13e721ad0> False
<tensorflow.py

In [18]:
# load data into dataframe with 'path' and 'target' columns
import pandas as pd
train_label_df = pd.read_csv('data.csv', delimiter=' ', header=1, names=["uid"])
train_label_df["path"] = train_label_df['uid'].apply(lambda x : 'data/' + x.split(',')[-1][1:].split('/')[-1])
train_label_df["target"] = train_label_df['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[2]))
train_label_df =train_label_df.drop(['uid'], axis=1)

train_label_df2 = pd.read_csv('data_temp.csv', delimiter=' ', header=1, names=["uid"])
train_label_df2["path"] = train_label_df2['uid'].apply(lambda x : "data/" + x.split(',')[0] + ".jpg")
train_label_df2["target"] = train_label_df2['uid'].apply(lambda x : int(x.split(',')[1])/int(x.split(',')[3]))
train_label_df2 =train_label_df2.drop(['uid'], axis=1)
#uid,likes,comments,followers
#B-A-bW9l1Gm,601,43,6715

In [22]:
train_df = pd.concat([train_label_df, train_label_df2], axis=0).reset_index().drop(['index'], axis=1)

In [23]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('data/') if isfile(join('data/', f))]

In [24]:
new_df = train_df
j = 0
for i in train_df['path']:
    #print(i.split("/")[-1])
    if i.split("/")[-1] not in onlyfiles:
        new_df = new_df.drop(train_df.index[train_df['path'] == i].tolist())
    else:
      j+=1
train_df = new_df.reset_index().drop(['index'], axis=1)
# All are found
train_df

,path,target
0,data/B9dZzI1n-EV.jpg,0.112025
1,data/B9cQMfVnkMe.jpg,0.638448
2,data/B9a0IQWnY9M.jpg,0.053981
3,data/B9NnTUrHoNX.jpg,0.143885
4,data/B9XuP83gx_j.jpg,0.030143
...,...,...
1429,data/B9M_-rNg0Er.jpg,0.289023
1430,data/B9WejSrBIsL.jpg,0.917219
1431,data/B9wPREWhq0l.jpg,0.128363
1432,data/B9Y_fqoHwUn.jpg,0.286299


In [25]:
batch_size = 128
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.2,
        rotation_range=5,
    
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col = 'path',
        y_col = 'target',
        # directory='data',
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='other',
       #  preprocessing_function=lambda x: x print(x)
)  # since we use binary_crossentropy loss, we need binary labels

Found 1434 validated image filenames.


In [26]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())  

['/device:CPU:0', '/device:XLA_CPU:0']


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=12800,
        epochs=50,
        #validation_data=validation_generator,
        #validation_steps=800 // batch_size)
        verbose = 2,
        # use_multiprocessing=False
)
model.save_weights('first_try.h5')